In [1]:
# Import modules
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import h5py
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
from sklearn.metrics import mean_squared_error, mean_absolute_error

# No_Overlapping_Window

In [2]:
path = './radar_images'
image = []
num_frame = 24
image_files = sorted(os.listdir(path))
num_samples = len(image_files) // num_frame
c = 0
for i in range(num_samples):
    sample = []
    for j in range(num_frame):
        file_index = i * num_frame + j
#         print("file_index-->", file_index)
        file_name = image_files[file_index]
#         print("file_name---->", file_name)
        if file_name.endswith('png'):
            img = load_img(path + '/' + file_name, color_mode='rgb')
            img = img_to_array(img)
            img = cv2.resize(img, (64,64))
            sample.append(img)
            c += 1
#         print("end of frame")
    image.append(sample)
#     print("end of sample")
print(c)
dataset = np.array(image)

print(dataset.shape)

9024
(376, 24, 64, 64, 3)


# Overlapping_Window

In [2]:
path = './radar_images'

image_files = sorted(os.listdir(path))
image = []

# Load the images.
for i in range(0, len(image_files) - 24, 1):
    window = image_files[i:i + 24]
    sample = []
    for file_name in window:
        if file_name.endswith('png'):
#             print(file_name)
            img = cv2.imread(path + '/' + file_name)
            img = cv2.resize(img, (64, 64))
            sample.append(img)
    image.append(sample)
dataset = np.array(image)
print(dataset.shape)

(9000, 24, 64, 64, 3)


In [5]:
# Split into train and validation sets using indexing to optimize memory.
indexes = np.arange(dataset.shape[0])
np.random.shuffle(indexes)
train_index = indexes[: int(0.7 * dataset.shape[0])]
val_index = indexes[int(0.7 * dataset.shape[0]):int(0.9 * dataset.shape[0])]
test_index = indexes[int(0.9 * dataset.shape[0]):]
train_dataset = dataset[train_index]
val_dataset = dataset[val_index]
test_dataset = dataset[test_index]

# Normalize the data to the 0-1 range.
train_dataset = train_dataset / 255
val_dataset = val_dataset / 255
test_dataset = test_dataset / 255

# Create input(x) and label(y)
def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 12, :, :]
    y = data[:, 12 : data.shape[1], :, :]
    return x, y


# Apply the processing function to the datasets.
x_train, y_train = create_shifted_frames(train_dataset)
x_val, y_val = create_shifted_frames(val_dataset)
x_test, y_test = create_shifted_frames(test_dataset)

# Inspect the dataset.
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))
print("Test Dataset Shapes: " + str(x_test.shape) + ", " + str(y_test.shape))

Training Dataset Shapes: (6300, 12, 64, 64, 3), (6300, 12, 64, 64, 3)
Validation Dataset Shapes: (1800, 12, 64, 64, 3), (1800, 12, 64, 64, 3)
Test Dataset Shapes: (900, 12, 64, 64, 3), (900, 12, 64, 64, 3)


In [4]:
# save the dataset
with h5py.File('./dataset/12framesRGB_overlap.h5', 'w') as hf:
    hf.create_dataset("train", data=train_dataset)
    hf.create_dataset("val", data=val_dataset)
    hf.create_dataset("test", data=test_dataset)

In [2]:
# Load the dataset
with h5py.File('./dataset/12framesRGB_overlap.h5', 'r') as hf:
    train_dataset = hf["train"][:]
    val_dataset = hf["val"][:]
    test_dataset = hf["test"][:]

# Create input(x) and label(y)
def create_shifted_frames(data):
    x = data[:, 0 : data.shape[1] - 12, :, :]
    y = data[:, 12 : data.shape[1], :, :]
    return x, y

# Apply the processing function to the datasets.
x_train, y_train = create_shifted_frames(train_dataset)
x_val, y_val = create_shifted_frames(val_dataset)
x_test, y_test = create_shifted_frames(test_dataset)

# Inspect the dataset.
print("Training Dataset Shapes: " + str(x_train.shape) + ", " + str(y_train.shape))
print("Validation Dataset Shapes: " + str(x_val.shape) + ", " + str(y_val.shape))
print("Test Dataset Shapes: " + str(x_test.shape) + ", " + str(y_test.shape))

In [4]:
# Creating output folder
output_dir = './output/WP_12framesconvLSTM_overlap_d3'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Building the model 

In [5]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(None, *x_train.shape[2:]))

x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    dilation_rate=(3, 3),
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=64,
    kernel_size=(3, 3),
    padding="same",
    dilation_rate=(3, 3),
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=128,
    kernel_size=(1, 1),
    padding="same",
    dilation_rate=(3, 3),
    return_sequences=True,
    activation="relu",
)(x)
x = layers.Conv3D(
    filters=3, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
)(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(
    loss=keras.losses.binary_crossentropy, 
    optimizer=keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy", "mean_squared_error", "mean_absolute_error"]
)
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


# Training the model

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)
# Define modifiable training hyperparameters.

epochs = 100
batch_size = 5

# Fit the model to the training data.
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)
model.save(os.path.join(output_dir, "WP_12framesconvLSTM_overlap_d2.h5"))

Train on 6300 samples, validate on 1800 samples


2024-10-19 10:26:39.431233: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2024-10-19 10:26:39.486451: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-19 10:26:39.488049: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Quadro P5000 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:02:00.0
2024-10-19 10:26:39.488139: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-19 10:26:39.488308: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Quadro P5000 major: 6 minor: 1 memoryClockRate(GHz): 1.7335
pciBusID: 0000:03:00.0
2024-

Epoch 1/100


2024-10-19 10:26:43.571260: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 679477248 exceeds 10% of system memory.
2024-10-19 10:26:43.571270: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 679477248 exceeds 10% of system memory.


   5/6300 [..............................] - ETA: 2:25:10 - loss: 0.6990 - acc: 0.4438 - mean_squared_error: 0.2416 - mean_absolute_error: 0.4828

2024-10-19 10:26:48.754015: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 679477248 exceeds 10% of system memory.
2024-10-19 10:26:48.754027: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 679477248 exceeds 10% of system memory.


  10/6300 [..............................] - ETA: 2:06:06 - loss: 0.6143 - acc: 0.6455 - mean_squared_error: 0.1990 - mean_absolute_error: 0.4312

2024-10-19 10:26:53.973913: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 679477248 exceeds 10% of system memory.


6300/6300 [==============================] - 6338s 1s/sample - loss: 0.1386 - acc: 0.8765 - mean_squared_error: 0.0288 - mean_absolute_error: 0.0694 - val_loss: 0.1289 - val_acc: 0.8826 - val_mean_squared_error: 0.0262 - val_mean_absolute_error: 0.0667
Epoch 2/100
6300/6300 [==============================] - 6324s 1s/sample - loss: 0.1285 - acc: 0.8795 - mean_squared_error: 0.0261 - mean_absolute_error: 0.0635 - val_loss: 0.1498 - val_acc: 0.8821 - val_mean_squared_error: 0.0262 - val_mean_absolute_error: 0.0574
Epoch 3/100
6300/6300 [==============================] - 6318s 1s/sample - loss: 0.1265 - acc: 0.8801 - mean_squared_error: 0.0256 - mean_absolute_error: 0.0623 - val_loss: 0.1277 - val_acc: 0.8832 - val_mean_squared_error: 0.0252 - val_mean_absolute_error: 0.0686
Epoch 4/100
6300/6300 [==============================] - 6319s 1s/sample - loss: 0.1249 - acc: 0.8805 - mean_squared_error: 0.0252 - mean_absolute_error: 0.0614 - val_loss: 0.1292 - val_acc: 0.8836 - val_mean_squared_

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Model Loss", fontsize=25)
plt.xlabel("Epoch", fontsize=20)
plt.ylabel("Loss", fontsize=20)
plt.legend(loc = "upper right", fontsize=15)
plt.savefig(os.path.join(output_dir, "WP_12framesconvLSTM_overlap_loss_d3.png"))

plt.figure(figsize=(10, 6))
plt.plot(history.history["acc"], label="Train Accuracy")
plt.plot(history.history["val_acc"], label="Validation Accuracy")
plt.title("Model Accuracy", fontsize=25)
plt.xlabel("Epoch", fontsize=20)
plt.ylabel("Accuracy", fontsize=20)
plt.legend(loc = "lower right", fontsize=15)
plt.savefig(os.path.join(output_dir, "WP_12framesconvLSTM_overlap_accuracy_d3.png"))

plt.figure(figsize=(10, 6))
plt.plot(history.history["mean_squared_error"], label="Train MSE")
plt.plot(history.history["val_mean_squared_error"], label="Validation MSE")
plt.title("Model Mean Squared Error", fontsize=25)
plt.xlabel("Epoch", fontsize=20)
plt.ylabel("Mean Squared Error", fontsize=20)
plt.legend(loc = "upper right", fontsize=15)
plt.savefig(os.path.join(output_dir, "WP_12framesconvLSTM_overlap_mse_d3.png"))

plt.figure(figsize=(10, 6))
plt.plot(history.history["mean_absolute_error"], label="Train MAE")
plt.plot(history.history["val_mean_absolute_error"], label="Validation MAE")
plt.title("Model Mean Absolute Error", fontsize=25)
plt.xlabel("Epoch", fontsize=20)
plt.ylabel("Mean Absolute Error", fontsize=20)
plt.legend(loc = "upper right", fontsize=15)
plt.savefig(os.path.join(output_dir, "WP_12framesconvLSTM_overlap_mae_d3.png"))

In [ ]:
avg_train_acc = np.mean(history.history["acc"])
avg_val_acc = np.mean(history.history["val_acc"])
avg_mse = np.mean(history.history["mean_squared_error"])
avg_val_mse = np.mean(history.history["val_mean_squared_error"])
avg_mae = np.mean(history.history["mean_absolute_error"])
avg_val_mae = np.mean(history.history["val_mean_absolute_error"])

print("Average Training Accuracy: " + str(avg_train_acc))
print("Average Validation Accuracy: " + str(avg_val_acc))
print("Average Training MSE: " + str(avg_mse))
print("Average Validation MSE: " + str(avg_val_mse))
print("Average Training MAE: " + str(avg_mae))
print("Average Validation MAE: " + str(avg_val_mae))


predictions = model.predict(x_test)
print(predictions.shape)
print("Number of predicted frames:" + str(len(predictions)))

# Testing the model

In [ ]:
# Select a few random examples from the dataset.
# example = test_dataset[np.random.choice(range(len(test_dataset)), size=1)[0]]
# print(example.shape)

# select a fixed example index
example_index = 13   # Change as needed
example = test_dataset[example_index]
print(example.shape)

# Pick the first/last ten frames from the example.
frames = example[:12, ...]
original_frames = example[12:, ...]

# predict the next 18 fames
new_prediction = model.predict(np.expand_dims(frames, axis=0))
new_prediction = np.squeeze(new_prediction, axis=0)

# Print the dimensions of your images
print("Original Frames Shape:", original_frames.shape)
print("New Prediction Shape:", new_prediction.shape)

# Reshape and preprocess your data if needed
new_prediction = new_prediction.reshape((-1, 64, 64, 1))
original_frames = original_frames.reshape((-1, 64, 64, 1))

# Initialize variables
batch_size = len(new_prediction)
mse_total = 0
mae_total = 0
pmae_total = 0

# Loop through each example in the batch
for i in range(batch_size):
    pmae = 0
    for j in range(12):  # Assuming you want to evaluate the first 10 frames
        mse_total += mean_squared_error(original_frames[i, j], new_prediction[i, j])
        mae_total += mean_absolute_error(original_frames[i, j], new_prediction[i, j])
        pmae += 1.0 - mean_absolute_error(original_frames[i, j], new_prediction[i, j])
    pmae_total += pmae * 100

# Calculate metrics
mse = mse_total / (batch_size * 12)
mae = mae_total / (batch_size * 12)
pmae_total = pmae_total / (batch_size * 12)

print('MSE: ', mse)
print('MAE: ', mae)
print('PMAE: ', pmae_total)

In [ ]:
# Select a few random examples from the dataset.
# examples = test_dataset[np.random.choice(range(len(test_dataset)), size=5)]

# Select fixed examples (e.g., first 5 samples)
# You can either use specific indices like:
fixed_indices = [1, 4, 11, 21, 25]  # Change as needed

examples = test_dataset[fixed_indices]  # Select those examples



# Iterate over the examples and predict the frames.
predicted_videos = []
for example in examples:
    # Pick the first/last ten frames from the example.
    frames = example[:12, ...]
    original_frames = example[12:, ...]
    new_predictions = np.zeros(shape=(12, *frames[0].shape))

    # Predict a new set of 10 frames.
    for i in range(12):
        # Extract the model's prediction and post-process it.
        frames = example[: 12 + i + 1, ...]
        new_prediction = model.predict(np.expand_dims(frames, axis=0))
        new_prediction = np.squeeze(new_prediction, axis=0)
        predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

        # Extend the set of prediction frames.
        new_predictions[i] = predicted_frame

    # Create and save GIFs for each of the ground truth/prediction images.
    for frame_set in [original_frames, new_predictions]:
        # Construct a GIF from the selected video frames.
        current_frames = np.squeeze(frame_set)
        current_frames = (current_frames * 255).astype(np.uint8)

        # Construct a GIF from the frames.
        with io.BytesIO() as gif:
            imageio.mimsave(gif, current_frames, "GIF", fps=5)
            predicted_videos.append(gif.getvalue())

# Display the videos.
print("Truth\tPrediction")
for i in range(0, len(predicted_videos), 2):
    # Construct and display an `HBox` with the ground truth and prediction.
    box = HBox(
        [
            widgets.Image(value=predicted_videos[i], layout=Layout(width='150px', height='150px')),
            widgets.Image(value=predicted_videos[i + 1], layout=Layout(width='150px', height='150px')),
        ]
    )
    display(box)

In [ ]:
from sklearn.metrics import mean_squared_error

# Select a random example from the validation dataset.
# example = test_dataset[np.random.choice(range(len(test_dataset)), size=1)[0]]

# Choose a specific example from the validation dataset (e.g., index 0)
example_index = 4  # Change this to the desired index
example = test_dataset[example_index]

# Pick the first/last ten frames from the example.
frames = example[:12, ...]
original_frames = example[12:, ...]

# Predict a new set of 10 frames.
for _ in range(12):
    # Extract the model's prediction and post-process it.
    new_prediction = model.predict(np.expand_dims(frames, axis=0))
    new_prediction = np.squeeze(new_prediction, axis=0)
    predicted_frame = np.expand_dims(new_prediction[-1, ...], axis=0)

    # Extend the set of prediction frames.
    frames = np.concatenate((frames, predicted_frame), axis=0)

 # Evaluate the model
# accuracy = mean_squared_error(original_frames, frames)

# # Print the evaluation metrics
# print(f"Accuracy: {accuracy}")


# Construct a figure for the original and new frames.
fig, axes = plt.subplots(2, 12, figsize=(20, 4))

# Plot the original frames.
for idx, ax in enumerate(axes[0]):
    ax.imshow(original_frames[idx])
    ax.set_title(f"Frame {idx + 13}")
    ax.axis("off")

# Plot the new frames.
new_frames = frames[12:, ...]
for idx, ax in enumerate(axes[1]):
    ax.imshow(new_frames[idx])
    ax.set_title(f"Frame {idx + 13}")
    ax.axis("off")

# Display the figure.
plt.show()